In [6]:
!pip install setuptools==70.0.0

  Using cached setuptools-70.0.0-py3-none-any.whl.metadata (5.9 kB)
Using cached setuptools-70.0.0-py3-none-any.whl (863 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 71.0.4
    Uninstalling setuptools-71.0.4:
      Successfully uninstalled setuptools-71.0.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.


In [7]:
!pip install crewai -r ../requirements.txt

  Using cached crewai-0.70.1-py3-none-any.whl.metadata (19 kB)
  Using cached uvicorn-0.30.1-py3-none-any.whl.metadata (6.3 kB)
  Using cached fastapi-0.110.3-py3-none-any.whl.metadata (24 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached crewai-0.51.1-py3-none-any.whl.metadata (14 kB)
  Using cached langchain-0.2.15-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_aws-0.1.17-py3-none-any.whl.metadata (3.2 kB)
  Using cached tiktoken-0.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
  Using cached pydantic-2.8.2-py3-none-any.whl.metadata (125 kB)
  Using cached durationpy-0.6-py3-none-any.whl.metadata (365 bytes)
  Using cached anthropic-0.36.1-py3-none-any.whl.metadata (21 kB)
  Using cached langgraph-0.2.38-py3-none-any.whl.metadata (13 kB)
  Using cached unstructured-0.15.14-py3-none-any.whl.metadata (29 kB)
  Using cached sentence_transformers-3.2.0-py3-none-any.whl.metadata (10 kB)
  Using cached app

In [10]:
import dotenv
assert dotenv.load_dotenv()

In [11]:
# Import required libraries
import os
from langchain_aws import ChatBedrock

# Set up the model ID for Claude
MODEL_ID3 = "meta.llama3-8b-instruct-v1:0"
MODEL_ID5 = "meta.llama3-70b-instruct-v1:0"
#MODEL_ID = "mistral.mistral-7b-instruct-v0:2"
MODEL_ID4 = "mistral.mixtral-8x7b-instruct-v0:1"
MODEL_ID2 = "anthropic.claude-3-haiku-20240307-v1:0"
MODEL_ID = "anthropic.claude-3-5-sonnet-20240620-v1:0"

# Initialize the ChatBedrock instance
llm = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0})
llm2 = ChatBedrock(model_id=MODEL_ID2, model_kwargs={'temperature': 0})
llm3 = ChatBedrock(model_id=MODEL_ID3, model_kwargs={'temperature': 0})
llm4 = ChatBedrock(model_id=MODEL_ID, model_kwargs={'temperature': 0.7})

In [12]:
from typing import Literal

from langchain_core.messages import AIMessage
from langchain_core.tools import tool

from langgraph.prebuilt import ToolNode
from langgraph.graph import StateGraph, MessagesState, START, END

### Data Engineer

In [42]:
from langchain_core.tools import tool
from sqlalchemy import text
from static.util import ENGINE
from typing import Literal


@tool
def query_database(query: str) -> str:
    """Query the PIRLS postgres database and return the results as a string.

    Args:
        query (str): The SQL query to execute.

    Returns:
        str: The results of the query as a string, where each row is separated by a newline.

    Raises:
        Exception: If the query is invalid or encounters an exception during execution.
    """
    # lower_query = query.lower()
    # record_limiters = ['count', 'where', 'limit', 'distinct', 'having', 'group by']
    # if not any(word in lower_query for word in record_limiters):
    #     return 'WARNING! The query you are about to perform has no record limitations! In case of large tables and ' \
    #            'joins this will return an incomprehensible output.'

    with ENGINE.connect() as connection:
        try:
            res = connection.execute(text(query))
        except Exception as e:
            return f'Wrong query, encountered exception {e}.'

    max_result_len = 3_000
    ret = '\n'.join(", ".join(map(str, result)) for result in res)
    if len(ret) > max_result_len:
        ret = ret[:max_result_len] + '...\n(results too long. Output truncated.)'

    return f'Query: {query}\nResult: {ret}'


@tool
def get_possible_answers_to_question(
        general_table: Literal['Students', 'Curricula', 'Homes', 'Teachers', 'Schools'],
        questionnaire_answers_table: Literal['StudentQuestionnaireAnswers', 'CurriculumQuestionnaireAnswers', 'HomeQuestionnaireAnswers', 'TeacherQuestionnaireAnswers', 'SchoolQuestionnaireAnswers'],
        questionnaire_entries_table: Literal['StudentQuestionnaireEntries', 'CurriculumQuestionnaireEntries', 'HomeQuestionnaireEntries', 'TeacherQuestionnaireEntries', 'SchoolQuestionnaireEntries'],
        question_code: str
) -> str:
    """Query the database and returns possible answer to a given question

    Args:
        general_table (str): the generic table related to the question topic. Can be one of: 'Students', 'Curricula', 'Homes', 'Teachers', 'Schools'
        questionnaire_answers_table (str): the table related to the `general_table` containing answers.
        questionnaire_entries_table (str): the table related to the `general_table` containing all possible questions.
        question_code (str): the code of the question the full list of possible answers to is returned.

    Returns:
        str: The list of all possible answers to the question with the code given in `question_code`.
    """
    entity_id = 'curriculum_id' if general_table.lower() == 'curricula' else f'{general_table.lower()[:-1]}_id'
    query = f"""
        SELECT DISTINCT ATab.Answer
        FROM {general_table} AS GTab
        JOIN {questionnaire_answers_table} AS ATab ON ATab.{entity_id} = GTab.{entity_id}
        JOIN {questionnaire_entries_table} AS ETab ON ETab.Code = ATab.Code
        WHERE ETab.Code = '{question_code.replace("'", "").replace('"', '')}'
    """

    with ENGINE.connect() as connection:
        try:
            res = connection.execute(text(query))
        except Exception as e:
            return f'Wrong query, encountered exception {e}.'

    ret = ""
    for result in res:
        ret += ", ".join(map(str, result)) + "\n"

    return ret


@tool
def get_questions_of_given_type(
    general_table: Literal['Students', 'Curricula', 'Homes', 'Teachers', 'Schools'],
    questionnaire_answers_table: Literal['StudentQuestionnaireAnswers', 'CurriculumQuestionnaireAnswers', 'HomeQuestionnaireAnswers', 'TeacherQuestionnaireAnswers', 'SchoolQuestionnaireAnswers'],
    questionnaire_entries_table: Literal['StudentQuestionnaireEntries', 'CurriculumQuestionnaireEntries', 'HomeQuestionnaireEntries', 'TeacherQuestionnaireEntries', 'SchoolQuestionnaireEntries'],
    question_type: str
) -> str:
    """Query the database and returns questions of a given type with their codes.

        Args:
            general_table (str): the generic table related to the question topic. Can be one of: 'Students', 'Curricula', 'Homes', 'Teachers', 'Schools'
            questionnaire_answers_table (str): the table related to the `general_table` containing answers.
            questionnaire_entries_table (str): the table related to the `general_table` containing all possible questions.
            question_type (str): the type of the question group.

        Returns:
            str: The list of all questions of type specified by `question_type`
        """
    entity_id = 'curriculum_id' if general_table.lower() == 'curricula' else f'{general_table.lower()[:-1]}_id'
    query = f"""
        SELECT DISTINCT ETab.Question, ETab.Code
        FROM {general_table} AS GTab
        JOIN {questionnaire_answers_table} AS ATab ON ATab.{entity_id} = GTab.{entity_id}
        JOIN {questionnaire_entries_table} AS ETab ON ETab.Code = ATab.Code
        WHERE ETab.Type = '{question_type.replace("'", "").replace('"', '')}'
    """

    with ENGINE.connect() as connection:
        try:
            res = connection.execute(text(query))
        except Exception as e:
            return f'Wrong query, encountered exception {e}.'

    questions = []
    for question, code in res:
        questions.append(f'(Code: {code}) {question}\n')
    return ''.join(questions)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [43]:
tools = [query_database, get_possible_answers_to_question, get_questions_of_given_type]
tool_node = ToolNode(tools)

In [44]:
model_with_tools = llm.bind_tools(tools)

In [45]:
def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


def call_model(state: MessagesState):
    messages = state["messages"]

    # Background information to guide the LLM's behavior
    system_message = {
        "role": "system",
        "content": (
            """
            Answer the following question:    
            {user_question}

            When applicable, search for relevant data in the PIRLS 2021 dataset.

            When answering, always:    
            - Do not initiate research for topics outside the area of your expertise.     
            - Ensure that your dataset queries are accurate and relevant to the research questions.
            - Unless instructed otherwise, explain how you come to your conclusions and provide evidence to support your claims with specific data.
            - Prioritize specific findings including numbers and percentages in line with best practices in statistics
            - Data and numbers should be provided in tables to increase readability.
            - Try to go the extra mile for open questions (e.g. correlate data with socioeconomic status, compare across countries within a region, integrate suggestions that you have into your query)

            expected_output:
            A complete answer to the scientific questions from the domain expert with additional context on correlations and causations.
            
            You are the Data Engineer for the PIRLS project. 
            You are an expert PostgreQSL user and have access to the full PIRLS 2021 dataset. 
            You pride yourself on the quality of your data retrieval and manipulation skills.

            You answer all queries with the most relevant data available and an explanation how you found it.
            You know that the database has millions of entries. Always limit your queries to return only the necessary data.
            If data is not provided in the dataset (e.g. trend data), stop the database search.
            Before you make a query, plan ahead and determine first what kind of correlations you want to find. 
            Reduce the amount of queries to the dataset as much as possible.
            NEVER return more than 100 rows of data.
            NEVER use the ROUND function. Instead use the CAST function for queries.
            For trend only rely on csv input. Don't try to merge the data with data from the database.
            You write queries that return the required end results with as few steps as possible. 
            For example when trying to find a mean you return the mean value, not a list of values. 

            Ensure that your results follow best practices in statistics (e.g. check for relevancy, percentiles).

            ### Trend data by country
            Trend data by country is stored as a csv under "trend_data/pirls_trends.csv". It uses ";" as a separator.

            ## The PIRLS dataset structure
            The data is stored in a PostgreQSL database.

            # Schema and explanation
            Students
            Student_ID: Int (Primary Key) - uniquely identifies student
            Country_ID: Int (Foreign Key) - uniquely identifies student's country
            School_ID: Int (Foreign Key) - uniquely identifies student's school
            Home_ID: Int (Foreign Key) - uniquely identifies student's home

            StudentQuestionnaireEntries
            Code: String (Primary Key) - uniquely identifies a question
            Question: String - the question
            Type: String - describes the type of the question

            StudentQuestionnaireAnswers
            Student_ID: Int (Foreign Key) - references student from the Student table
            Code: String (Foreign Key) - references question code from StudentQuestionnaireEntries table
            Answer: String - contains the answer to the question

            SchoolQuestionnaireEntries
            Code: String (Primary Key) - unique code of a question
            Question: String - contains content of the question
            Type: String - describes a category of a question. There are several questions in each category. The categories are: Instructional Time, Reading in Your School, School Emphasis on Academic Success, School Enrollment and Characteristics, Students’ Literacy Readiness, Principal Experience and Education, COVID-19 Pandemic, Resources and Technology, School Discipline and Safety

            SchoolQuestionnaireAnswers
            School_ID: Int (Composite Key) - references school from Schools table
            Code: String (Composite Key) - references score code from SchoolQuestionnaireEntries table
            Answer: String - answer to the question from the school

            TeacherQuestionnaireEntries
            Code: String (Primary Key)
            Question: String
            Type: String

            TeacherQuestionnaireAnswers
            Teacher_ID: Int (Foreign Key) - references teacher from Teachers table
            Code: String (Foreign Key) - references score code from TeacherQuestionnaireEntries table
            Answer: String - answer to the question from the teacher

            HomeQuestionnaireEntries
            Code: String (Primary Key)
            Question: String
            Type: String

            HomeQuestionnaireAnswers
            Home_ID: Int (Foreign Key)
            Code: String (Foreign Key)
            Answer: String

            CurriculumQuestionnaireEntries
            Code: String (Primary Key)
            Question: String
            Type: String

            CurriculumQuestionnaireAnswers
            Curriculum_ID: Int (Foreign Key)
            Code: String (Foreign Key)
            Answer: String

            Schools
            School_ID: Int (Primary Key) - uniquely identifies a School
            Country_ID: Int (Foreign Key) - uniquely identifies a country

            Teachers
            Teacher_ID: Int (Primary Key) - uniquely identifies a Teacher
            School_ID: Int (Foreign Key) - uniquely identifies a School

            StudentTeachers
            Teacher_ID: Int (Foreign Key)
            Student_ID: Int (Foreign Key)

            Homes
            Home_ID: Int (Primary Key) - uniquely identifies a Home

            Curricula
            Curriculum_ID: Int (Primary Key)
            Country_ID: Int (Foreign Key)

            StudentScoreEntries
            Code: String (Primary Key) - See below for examples of codes
            Name: String
            Type: String

            StudentScoreResults
            Student_ID: Int (Foreign Key) - references student from Students table
            Code: String (Foreign Key) - references score code from StudentScoreEntries table
            Score: Float - the numeric score for a student

            Benchmarks
            Benchmark_ID: Int (Primary Key) - uniquely identifies benchmark
            Score: Int - the lower bound of the benchmark. Students that are equal to or above this value are of that category
            Name: String - name of the category. Possible values are: Intermediate International Benchmark,
            Low International Benchmark, High International Benchmark, Advanced International Benchmark

            Countries
            Country_ID: Int (Primary Key) - uniquely identifies a country
            Name: String - full name of the country
            Code: String - 3 letter code of the country
            Benchmark: Boolean - boolean value saying if the country was a benchmark country. 
            TestType: String - describes the type of test taken in this country. It's either digital or paper.

            # Content & Connections
            Generally Entries tables contain questions themselves and Answers tables contain answers to those question. 
            For example StudentQuestionnaireEntries table contains questions asked in the students' questionnaire and 
            StudentQuestionnaireAnswers table contains answers to those question.

            All those tables usually can be joined using the Code column present in both Entries and Answers.

            Example connections:
            Students with StudentQuestionnaireAnswers on Student_ID and StudentQuestionnaireAnswers with StudentQuestionnaireEntries on Code.
            Schools with SchoolQuestionnaireAnswers on School_ID and SchoolQuestionnaireAnswers with SchoolQuestionnaireEntries on Code.
            Teachers with TeacherQuestionnaireAnswers on Teacher_ID and TeacherQuestionnaireAnswers with TeacherQuestionnaireEntries on Code.
            Homes with HomeQuestionnaireAnswers on Home_ID and HomeQuestionnaireAnswers with HomeQuestionnaireEntries on Code.
            Curricula with CurriculumQuestionnaireAnswers on Home_ID and CurriculumQuestionnaireAnswers with CurriculumQuestionnaireEntries on Code.

            In the student evaluation process 5 distinct scores were measured. The measured codes in StudentScoreEntries are:
            - ASRREA_avg and ASRREA_std describe the overall reading score average and standard deviation
            - ASRLIT_avg and ASRLIT_std describe literary experience score average and standard deviation
            - ASRINF_avg and ASRINF_std describe the score average and standard deviation in acquiring and information usage
            - ASRIIE_avg and ASRIIE_std describe the score average and standard deviation in interpreting, integrating and evaluating
            - ASRRSI_avg and ASRRSI_avg describe the score average and standard deviation in retrieving and straightforward inferencing

            Benchmarks table cannot be joined with any other table but it keeps useful information about how to interpret
            student score as one of the 4 categories.   

            # Example Approach for retrieving COVID-19 related data
            1. **Retrieve COVID-19 Related Records**: Start by fetching entries from the `School Questionnaire Entries` table that are associated with the **COVID-19 Pandemic**. This step helps narrow down our focus to pandemic-related questions.
            2. **Identify the Relevant Question Code**: After reviewing the questions retrieved in the previous step, identify and note the **Code** for the question that inquires about the number of weeks normal primary school operations were impacted by the COVID-19 Pandemic. This code will be crucial for filtering responses in the next steps.
            3. **Filter Responses by Question Code**: With the question code in hand, proceed to filter records in the `School Questionnaire Answers` table. Ensure you're only selecting entries that respond to our identified question regarding the pandemic's impact on school operations.
            4. **Extract Unique Answers**: For the final step, refine your query to return only distinct values of **Answer** column from the filtered responses. This will provide a clear view of all unique answers given to the question, offering insights into the varied impacts of the pandemic on schools.

            # Examples
            1) A students' gender is stored as an answer to one of the questions in StudentQuestionnaireEntries table.
            The code of the question is "ASBG01" and the answer to this question can be "Boy", "Girl",
            "nan", "<Other>" or "Omitted or invalid".

            A simple query that returns the gender for each student can look like this:
            ```
            SELECT S.Student_ID,
               CASE 
                   WHEN SQA.Answer = 'Boy' THEN 'Male'
                   WHEN SQA.Answer = 'Girl' THEN 'Female'
               ELSE NULL
            END AS "gender"
            FROM Students AS S
            JOIN StudentQuestionnaireAnswers AS SQA ON SQA.Student_ID = S.Student_ID
            JOIN StudentQuestionnaireEntries AS SQE ON SQE.Code = SQA.Code
            WHERE SQA.Code = 'ASBG01'
            ```

            2) A simple query that answers the question 'What percentage of students in Egypt reached the Low International Benchmark?' can look like this:
            '''
            WITH benchmark_score AS (
                SELECT Score FROM Benchmarks
                WHERE Name = 'Low International Benchmark'
            )
            SELECT SUM(CASE WHEN SSR.score >= bs.Score THEN 1 ELSE 0 END) / COUNT(*)::float as percentage
            FROM Students AS S
            JOIN Countries AS C ON C.Country_ID = S.Country_ID
            JOIN StudentScoreResults AS SSR ON SSR.Student_ID = S.Student_ID
            CROSS JOIN benchmark_score AS bs
            WHERE C.Name = 'Egypt' AND SSR.Code = 'ASRREA_avg'
            '''

            3) A simple query that answers the question 'Which country had an average reading score between 549 and 550 for its students?' can look like this:
            '''
            SELECT C.Name AS Country
            FROM Students as S
            JOIN Countries as C ON S.Country_ID = C.Country_ID
            JOIN StudentScoreResults SSR ON S.Student_ID = SSR.Student_ID
            WHERE SSR.Code = 'ASRREA_avg'
            GROUP BY C.Name
            HAVING AVG(ssr.Score) BETWEEN 549 AND 550;
            '''
            """
        )
    }

    # Add the system message at the start of the conversation
    messages = [system_message] + messages

    # Pass the messages to the model
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")

app = workflow.compile()

In [54]:
# example with a single tool call
for chunk in app.stream(
    {"messages": [("human", "What are the average reading scores for the top 10 countries? Please consider weighted results (e.g. TOTWGT).")]}, stream_mode="values"
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

What are the average reading scores for the top 10 countries? Please consider weighted results (e.g. TOTWGT).
================================== Ai Message ==================================
Tool Calls:
  query_database (toolu_bdrk_0119h4oj6v4qo4tA53gLgqtR)
 Call ID: toolu_bdrk_0119h4oj6v4qo4tA53gLgqtR
  Args:
    query: WITH WeightedScores AS (
    SELECT 
        C.Name AS Country,
        SUM(SSR.Score * SQA.Answer::float) / SUM(SQA.Answer::float) AS WeightedAvgScore
    FROM 
        Students S
    JOIN 
        Countries C ON S.Country_ID = C.Country_ID
    JOIN 
        StudentScoreResults SSR ON S.Student_ID = SSR.Student_ID
    JOIN 
        StudentQuestionnaireAnswers SQA ON S.Student_ID = SQA.Student_ID
    WHERE 
        SSR.Code = 'ASRREA_avg'
        AND SQA.Code = 'TOTWGT'
    GROUP BY 
        C.Name
)
SELECT 
    Country,
    CAST(WeightedAvgScore AS DECIMAL(10,2)) AS WeightedAvgScore
FROM

### Data Visualization

In [88]:
from langchain_core.tools import tool
import requests

@tool
def create_quickchart_url(
    chart_input: dict
) -> str:
    """
    Sends a POST request to the QuickChart API (https://quickchart.io/chart) to generate a chart, and returns the URL for the created chart.

    Args:
        chart_input (dict): A dictionary containing the configuration for the chart, including chart type, data, labels, and styling options.

    Returns:
        str: The URL of the generated chart if the request is successful.
             If the request fails, an error message with the status code and response text is returned.

    Example of `chart_input`:
        {
            "format": "svg",  # Specifies the image format (e.g., 'png' or 'svg')
            "chart": {
                "type": "bar",  # Type of the chart, such as 'bar', 'line', or 'pie'
                "data": {
                    "labels": ["Income Level", "Parental Education", "School Funding"],  # X-axis labels for chart categories
                    "datasets": [
                        {
                            "label": "Low Performance",  # Dataset label for low performance group
                            "data": [60, 65, 58],  # Corresponding values for the low performance group
                            "backgroundColor": "#DA9A8B"  # Color for the dataset bar (red)
                        },
                        {
                            "label": "Medium Performance",  # Dataset label for medium performance group
                            "data": [75, 78, 76],  # Corresponding values for the medium performance group
                            "backgroundColor": "#DCBB7C"  # Color for the dataset bar (orange)
                        },
                        {
                            "label": "High Performance",  # Dataset label for high performance group
                            "data": [90, 88, 85],  # Corresponding values for the high performance group
                            "backgroundColor": "#4FB293"  # Color for the dataset bar (green)
                        }
                    ]
                },
                "options": {
                    "title": {
                        "display": True,
                        "text": "Reading Scores vs Socioeconomic Factors"  # Chart title
                    },
                    "scales": {
                        "xAxes": [{
                            "scaleLabel": {
                                "display": True,
                                "labelString": "Socioeconomic Factors"  # Label for the x-axis
                            }
                        }],
                        "yAxes": [{
                            "scaleLabel": {
                                "display": True,
                                "labelString": "Reading Scores"  # Label for the y-axis
                            }
                        }]
                    },
                    "legend": {
                        "display": True,  # Determines if the legend should be displayed
                        "position": "bottom"  # Position of the legend
                    }
                }
            }
        }

    Example usage:
        create_quickchart_url(chart_input)

    """
    api_url = 'https://quickchart.io/chart/create'

    try:
        # Send POST request with the chart input
        response = requests.post(api_url, json=chart_input, timeout=10)
        response.raise_for_status()  # Raise HTTPError if the response status code is 4xx or 5xx

        # Parse the response JSON and extract the chart URL
        result = response.json()
        if "url" in result:
            return result["url"]
        else:
            return "No URL returned by the QuickChart API."

    except requests.exceptions.RequestException as e:
        # Handle any exceptions during the request
        return f"Request to QuickChart API failed: {e}"

In [89]:
tools = [create_quickchart_url]
tool_node = ToolNode(tools)

In [90]:
model_with_tools = llm.bind_tools(tools)

In [91]:
def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


def call_model(state: MessagesState):
    messages = state["messages"]

    # Background information to guide the LLM's behavior
    system_message = {
        "role": "system",
        "content": (
            """
            Provide a link to a visualization that helps to answer the following question:    
            {user_question}

            You are an expert in creating compelling and accurate data visualizations for the Progress in International Reading Literacy Study (PIRLS) project.
            Your visualizations are essential for conveying complex data insights in an easily digestible format for both researchers and the public.
            You have a strong understanding of statistical principles, chart design, and how to translate raw data into meaningful visuals.
            You work closely with the data engineer, writer, and other team members to ensure that the visualizations complement the research findings and provide added value.
            You thrive on precision, and you take pride in transforming numbers and datasets into clear, actionable visual stories.
            ALWAYS ensure the visualizations are easy to interpret and align with the overall research narrative.
            ALWAYS consider the audience when selecting the type of visualization, focusing on clarity and simplicity.
            ONLY reply with the url for the visualization.
            
            Create a visual representation of the data related to the most important research finding:
    
            The visualization should aim to provide clear insights into the dataset, making complex patterns, trends, or comparisons easy to understand.

            When creating the visualization, always:
            - Ensure the visual aligns with the overall research narrative and conclusions.
            - Choose the most appropriate chart type (e.g., bar chart, line graph, heat map) for the data presented.
            - Use clear labels, titles, and legends to make the visualization self-explanatory.
            - Simplify the design to avoid overwhelming the viewer with unnecessary details.
            """
        )
    }

    # Add the system message at the start of the conversation
    messages = [system_message] + messages

    # Pass the messages to the model
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")

app = workflow.compile()

In [92]:
# example with a single tool call
for chunk in app.stream(
    {"messages": [("human", "Which country had a reading score closest to 547 for fourth-grade students in the PIRLS 2021 study?")]}, stream_mode="values"
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

Which country had a reading score closest to 547 for fourth-grade students in the PIRLS 2021 study?
================================== Ai Message ==================================
Tool Calls:
  create_quickchart_url (toolu_bdrk_017nwFbQ5CNhT8SdKEw6EpLD)
 Call ID: toolu_bdrk_017nwFbQ5CNhT8SdKEw6EpLD
  Args:
    chart_input: {'format': 'png', 'chart': {'type': 'bar', 'data': {'labels': ['Singapore', 'Hong Kong SAR', 'Russia', 'Taiwan', 'England', 'Finland', 'Poland', 'Sweden', 'Bulgaria', 'Norway', 'Italy', 'Latvia', 'Hungary', 'Lithuania', 'Australia', 'Czech Republic', 'Austria', 'Denmark', 'Germany', 'Slovenia', 'Canada', 'Croatia', 'Ireland', 'Slovak Republic', 'Israel', 'Portugal', 'Spain', 'Northern Ireland', 'New Zealand', 'France', 'Belgium (Flemish)', 'United Arab Emirates', 'Bahrain', 'Albania', 'Malta', 'Cyprus', 'Georgia', 'North Macedonia', 'Montenegro', 'Serbia', 'Azerbaijan', 'Saudi Arabia',

In [85]:
chart_input = {'type': 'bar', 'data': {'labels': ['Hungary', 'Taiwan', 'Sweden', 'Dubai, UAE', 'Finland'], 'datasets': [{'label': 'Average Reading Score', 'data': [545.75, 548.27, 545.72, 545.44, 549.94], 'backgroundColor': 'rgba(54, 162, 235, 0.8)'}, {'label': 'Target Score (547)', 'data': [547, 547, 547, 547, 547], 'type': 'line', 'borderColor': 'rgba(255, 99, 132, 1)', 'borderWidth': 2, 'fill': False}]}, 'options': {'title': {'display': True, 'text': 'Countries with Reading Scores Closest to 547 in PIRLS 2021'}, 'scales': {'yAxes': [{'ticks': {'beginAtZero': False, 'min': 540, 'max': 555}, 'scaleLabel': {'display': True, 'labelString': 'Reading Score'}}], 'xAxes': [{'scaleLabel': {'display': True, 'labelString': 'Country'}}]}, 'legend': {'display': True, 'position': 'bottom'}}}


In [86]:
url = create_quickchart_url(chart_input)

In [87]:
url

"Request to QuickChart API failed: HTTPSConnectionPool(host='quickchart.io', port=443): Read timed out. (read timeout=10)"

### UNESCO API

In [71]:
from langchain_core.tools import tool
import requests


@tool
def get_unesco_data(indicators: list, geo_units: list, start: str = '2021', end: str = '2021', indicator_metadata: bool = False) -> dict:
    """
    Sends a GET request to the UNESCO API (https://api.uis.unesco.org/api/public) to retrieve data for multiple indicators.

    Args:
        indicators (list): A list of indicator codes to query.
        geo_units (list): A list of geographic units (countries) to include in the query.
        start (str): The start year for the data query. Defaults to '2021'.
        end (str): The end year for the data query. Defaults to '2021'.
        indicator_metadata (bool, optional): Whether to include indicator metadata in the response. Defaults to False.

    Returns:
        dict: The JSON response from the UNESCO API if the request is successful.
              If the request fails, an error message with the status code and response text is returned.

    Example usage:
        get_unesco_data(indicators=['XGDP.FSGOV', 'XGDP.EDU'], geo_units=['BRA', 'USA', 'DEU'])
    """
    base_url = 'https://api.uis.unesco.org/api/public/data/indicators'
    params = {
        'start': start,
        'end': end,
        'indicatorMetadata': str(indicator_metadata).lower()
    }

    # Add indicator parameters
    for indicator in indicators:
        params.setdefault('indicator', []).append(indicator)

    # Add geoUnit parameters
    for geo_unit in geo_units:
        params.setdefault('geoUnit', []).append(geo_unit)

    try:
        # Send GET request with the specified parameters
        response = requests.get(base_url, params=params, timeout=10)
        response.raise_for_status()  # Raise HTTPError if the response status code is 4xx or 5xx

        # Parse the response JSON
        return response.json()

    except requests.exceptions.RequestException as e:
        # Handle any exceptions during the request
        return {"error": f"Request to UNESCO API failed: {e}"}

In [72]:
tools = [get_unesco_data]
tool_node = ToolNode(tools)

In [73]:
model_with_tools = llm.bind_tools(tools)

In [78]:
def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


def call_model(state: MessagesState):
    messages = state["messages"]

    # Background information to guide the LLM's behavior
    system_message = {
        "role": "system",
        "content": (
            """
            Answer the following question:    
            {user_question}

            When applicable, search for relevant data in the UNESCO API.

            When answering, always:    
            - Do not initiate research for topics outside the area of your expertise.     
            - Ensure that your dataset queries are accurate and relevant to the research questions.
            - Unless instructed otherwise, explain how you come to your conclusions and provide evidence to support your claims with specific data.
            - Relevant words should be highlighted in the output.
            - Cite https://data.uis.unesco.org/ as a source within the text as a footnote.
            - Prioritize specific findings including numbers and percentages in line with best practices in statistics
            - Data and numbers should be provided in tables to increase readability.
            - Prioritize your findings based on correlation
            - ALWAYS limit your output to the most important finding (1 paragraph). Keep it short!
            - 
            
            expected_output:
            A complete answer to the scientific questions from the domain expert with additional context on correlations and causations.
            
            ## RELEVANT INDICATORS
            CR.1,"Completion rate, primary education, both sexes (%)"
            XGDP.FSGOV,"Government expenditure on education as a percentage of GDP (%)"
            XGDP.FSHH.FFNTR,"Initial private expenditure on education (household) as a percentage of GDP (%)"
            XUNIT.GDPCAP.1.FSGOV.FFNTR,"Initial government funding per primary student as a percentage of GDP per capita"
            XUNIT.GDPCAP.02.FSGOV.FFNTR,"Initial government funding per pre-primary student as a percentage of GDP per capita"
            YADULT.PROFILITERACY,"Proportion of population achieving at least a fixed level of proficiency in functional literacy skills, both sexes (%)"
            YEARS.FC.COMP.02,"Number of years of compulsory pre-primary education guaranteed in legal frameworks"
            YEARS.FC.COMP.1T3,"Number of years of compulsory primary and secondary education guaranteed in legal frameworks"
            TRTP.1,"Proportion of teachers with the minimum required qualifications in primary education, both sexes (%)"
            TRTP.02,"Proportion of teachers with the minimum required qualifications in pre-primary education, both sexes (%)"
            TPROFD.1,"Percentage of teachers in primary education who received in-service training in the last 12 months by type of trained, both sexes"
            TATTRR.1,"Teacher attrition rate from primary education, both sexes (%)"
            SCHBSP.1.WINFSTUDIS,"Proportion of primary schools with access to adapted infrastructure and materials for students with disabilities (%)"
            SCHBSP.1.WINTERN,"Proportion of primary schools with access to Internet for pedagogical purposes (%)"
            SCHBSP.1.WCOMPUT,"Proportion of primary schools with access to computers for pedagogical purposes (%)"
            SCHBSP.1.WELEC,"Proportion of primary schools with access to electricity (%)"
            ROFST.1.GPIA.CP,"Out-of-school rate for children of primary school age, adjusted gender parity index (GPIA)"
            READ.PRIMARY.LANGTEST,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, spoke the language of the test at home, both sexes (%)"
            READ.PRIMARY,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, both sexes (%)"
            PREPFUTURE.1.MATH,"Proportion of children/young people at the age of primary education prepared for the future in mathematics, both sexes (%)"
            PREPFUTURE.1.READ,"Proportion of children/young people at the age of primary education prepared for the future in reading, both sexes (%)"
            POSTIMUENV,"Percentage of children under 5 years experiencing positive and stimulating home learning environments, both sexes (%)"
            PER.BULLIED.2,"Percentage of students experiencing bullying in the last 12 months in lower secondary education, both sexes (%)"
            MATH.PRIMARY,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in mathematics, both sexes (%)"
            LR.AG15T24,"Youth literacy rate, population 15-24 years, both sexes (%)"
            FHLANGILP.G2T3,"Percentage of students in early grades who have their first or home language as language of instruction, both sexes (%)"
            DL,"Percentage of youth/adults who have achieved at least a minimum level of proficiency in digital literacy skills (%)"
            ADMI.ENDOFPRIM.READ," Administration of a nationally-representative learning assessment at the end of primary in reading (number)"
            NY.GDP.MKTP.CD,"GDP (current US$)"
            NY.GDP.PCAP.CD,"GDP per capita (current US$)"
            READ.G2.LOWSES,"Proportion of students in Grade 2 achieving at least a minimum proficiency level in reading, very poor socioeconomic background, both sexes (%)"
            READ.PRIMARY.RURAL,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, rural areas, both sexes (%)"
            READ.PRIMARY.URBAN,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, urban areas, both sexes (%)"
            READ.PRIMARY.WPIA,"Proportion of students at the end of primary education achieving at least a minimum proficiency level in reading, adjusted wealth parity index (WPIA)"
            
            """
        )
    }

    # Add the system message at the start of the conversation
    messages = [system_message] + messages

    # Pass the messages to the model
    response = model_with_tools.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(MessagesState)

# Define the two nodes we will cycle between
workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", should_continue, ["tools", END])
workflow.add_edge("tools", "agent")

app = workflow.compile()

In [79]:
# example with a single tool call
for chunk in app.stream(
    {"messages": [("human", "What are the main drivers for reading performance in DEU?")]}, stream_mode="values"
):
    chunk["messages"][-1].pretty_print()

================================ Human Message =================================

What are the main drivers for reading performance in DEU?
================================== Ai Message ==================================
Tool Calls:
  get_unesco_data (toolu_bdrk_01AmBCy36yE3uVAP56P2v7Qr)
 Call ID: toolu_bdrk_01AmBCy36yE3uVAP56P2v7Qr
  Args:
    indicators: ['READ.PRIMARY', 'XGDP.FSGOV', 'TRTP.1', 'SCHBSP.1.WINTERN', 'SCHBSP.1.WCOMPUT', 'POSTIMUENV', 'XUNIT.GDPCAP.1.FSGOV.FFNTR']
    geo_units: ['DEU']
    start: 2015
    end: 2021
================================= Tool Message =================================
Name: get_unesco_data

{"hints": [], "records": [{"indicatorId": "READ.PRIMARY", "geoUnit": "DEU", "year": 2016, "value": 94.50744628672335, "magnitude": null, "qualifier": null}, {"indicatorId": "READ.PRIMARY", "geoUnit": "DEU", "year": 2021, "value": 93.5873729906259, "magnitude": null, "qualifier": null}, {"indicatorId": "SCHBSP.1.WCOMPUT", "geoUnit": "DEU", "year": 2015, "val

In [59]:
import pandas as pd

df = pd.DataFrame(response)
df

,id,name,type,regionGroup
0,ABW,Aruba,NATIONAL,NaN
1,AFG,Afghanistan,NATIONAL,NaN
2,AGO,Angola,NATIONAL,NaN
3,AIA,Anguilla,NATIONAL,NaN
4,ALA,Åland Islands,NATIONAL,NaN
...,...,...,...,...
470,WB: Sub-Saharan Africa (IDA & IBRD),Sub-Saharan Africa (IDA & IBRD),REGIONAL,WB
471,WB: Sub-Saharan Africa (excluding high income),Sub-Saharan Africa (excluding high income),REGIONAL,WB
472,WB: Upper middle income (July 2023),Upper middle income (July 2023),REGIONAL,WB
473,WB: Upper middle income (July 2024),Upper middle income (July 2024),REGIONAL,WB


In [60]:
pirls_2021_participants = [
    "Australia", "Austria", "Azerbaijan", "Bahrain", "Belgium (Flemish)", "Belgium (French)", "Bulgaria", 
    "Canada", "Chile", "Chinese Taipei", "Croatia", "Cyprus", "Czech Republic", "Denmark", "Egypt", 
    "England", "Finland", "France", "Georgia", "Germany", "Hong Kong SAR", "Hungary", "Iran", "Ireland", 
    "Israel", "Italy", "Japan", "Kazakhstan", "Kuwait", "Latvia", "Lithuania", "Macao SAR", "Malta", 
    "Morocco", "Netherlands", "New Zealand", "North Macedonia", "Norway", "Oman", "Poland", "Portugal", 
    "Qatar", "Russian Federation", "Saudi Arabia", "Serbia", "Singapore", "Slovak Republic", "Slovenia", 
    "South Africa", "Spain", "Sweden", "Turkey", "United Arab Emirates", "United States", "Uzbekistan"
]

In [66]:
df[df["name"].isin(pirls_2021_participants)][["id", "name"]].reset_index(drop=True)

,id,name
0,ARE,United Arab Emirates
1,AUS,Australia
2,AUT,Austria
3,AZE,Azerbaijan
4,BGR,Bulgaria
5,BHR,Bahrain
6,CAN,Canada
7,CHL,Chile
8,CYP,Cyprus
9,DEU,Germany
